# Writing an Application

## Applications

In a blockchain application, the blockchain will store the state of the system, in addition to the immutable record of transactions that created that state. A client application will be used to send transactions to the blockchain. The smart contracts will encode some (if not all) of the business logic.

## Designing an Application Using the Javascript SDK

We will be looking at a sample blockchain application that interfaces with Hyperledger Sawtooth. This application was written by Zac Delventhal, a maintainer on Hyperledger Sawtooth, and was originally presented at Midwest JS 2017. This example is meant to introduce you to writing an application that interfaces with Hyperledger Sawtooth, as opposed to creating a complete production implementation.<br>

Hyperledger Sawtooth offers an SDK to abstract away many of the complicated aspects of blockchain technology. **SDKs are available in multiple languages, including Java, Python, and Javascript**. 
* In this example, we will be working with the **Javascript SDK**. We will be examining a web client and transaction processor putting our demonstrated scenario into action.

## Designing an Application
### Alexandra Groetsema

* [VIDEO](https://youtu.be/ZlXh6riRKZs)

> "Building an application requires a few important steps:<br>
first, **defining assets that will reside in the distributed ledger, as well as transactions that will act on these assets to change their state**;"<br><br>
"second, **designing transaction logic that operates on these assets**."<br><br>
"Now, as transactions are received by a Sawtooth node, they are forwarded to other nodes. A node is selected by a consensus model to publish a block. The block will contain any transactions that have been received and executed successfully."<br><br>
"The block is then broadcasted to the publishing nodes. Each sawtooth node receives a published block and verifies that that block is valid. It then notifies our application of any state changes."

## Review of Hyperledger Sawtooth Components

### Transaction validators 
validate transactions.

### Transaction families 
consist of "a group of operations or transaction types" (Dan Middleton) that are allowed on the shared ledger. Transaction families consist of both transaction processors (the server-side logic) and clients (for use from Web or mobile applications).

### The transaction processor 
provides the server-side business logic that operates on assets within a network.

### Transaction batches
are clusters of transactions that are either all committed to state, or are all not committed to state.

### The network layer
is responsible for communicating between validators in a Hyperledger Sawtooth network, including performing initial connectivity, peer discovery, and message handling.

### The global state
contains the current state of the ledger and a chain of transaction invocations. The state for all transaction families is represented on each validator. The state is split into namespaces, which allow flexibility for transaction family authors to define, share, and reuse global state data between transaction processors.

## Sawtooth Tuna Application

Let’s get our feet wet with an example of a simple Hyperledger Sawtooth blockchain application, written in JavaScript, that relates to the tuna fish supply scenario we discussed in our demonstrated scenario. The Sawtooth Tuna Chain allows a user to create named assets (tuna fish), and transfer them between different holders designated by a public key.<br>

In our example, we will look at:
* A transaction processor
* A simple browser-based client.

The transaction processor interfaces with a Sawtooth validator, and handles the validation of transactions.<br>

The client is a simple browser-based user interface, and will allow a user to manage public/private key pairs, and submit transactions using the Sawtooth REST API.<br>

Just in case you have not done so, clone the educational repository for this course with the Sawtooth Tuna Chain application code we have provided. Navigate in your terminal window to your projects folder or desktop.

```
$ git clone https://github.com/hyperledger/education.git
$ cd education
$ cd LFS171x
$ cd sawtooth-material
```

## File Structure of Application

Here you can see the file structure of the Sawtooth application:

![sawtooth-example-file-structure](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/d99202fc5d73b815887148f17dc52af4/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/File_Structure_of_Sawtooth_Application.png)

## Installation

Make sure you have Docker running on your machine before you run the next command. If you do not have Docker installed, you should review Chapter 4, Technical Requirements.<br>

* **Note**: Make sure you are in the **sawtooth-material** folder

We will use the provided docker-compose file to spin up some default Sawtooth components, including a validator and a REST API. Run the following command to start Sawtooth up:

```
$ docker-compose -f sawtooth-default.yaml up
```

* **Note**: If you are getting an error, you may need to run the following instead:

```
$ docker-compose -force sawtooth-default.yaml up
```

Let’s run the following commands to install dependencies for the transaction processor:

```
$ cd sawtooth-tuna
$ cd processor
$ npm install
```

Then, we will navigate to the client folder, and run these commands to install dependencies for and build the client:

```
$ cd ..
$ cd client
$ npm install
$ npm run build

$ cd ..   
```
* **Note**: Make sure you are in the sawtooth-tuna folder.

## Transaction Processor

To review, there are two main components of a transaction processor: a processor class and a handler class. The Javascript SDK offers a general-purpose processor class. The handler class is customized to the application, and holds the business logic for transactions. Usually, there is one transaction processor per use case.<br>

Make sure you are in the **sawtooth-tuna** folder.<br>

In a new terminal window, start up the transaction processor by running the following:

```
$ cd processor
$ npm start
```

## Browser Client

Start the client simply by opening `../client/index.html` in any browser window of your choice. One way of doing so is simply to open up the `education` folder and navigate to `sawtooth-tuna`. Click on the client folder and drag the `index.html` folder into a browser window.

## Creating a User

We are now ready to test out our application through the user interface. We have four options:
* Create a user within the supply chain
* Create a record of a tuna catch
* Transfer a tuna
* Accept or reject a transfer.

The application logic is contained mainly within the **handlers.js** file within the **processor** folder.<br>

Users are just public/private key pairs stored in **localStorage**.

```
makePrivateKey: () => {

let privateKey

do privateKey = randomBytes(32)

while (!secp256k1.privateKeyVerify(privateKey))

return privateKey.toString('hex')

}
```

This function creates a random 256-bit private key represented as a 64-char hex string on the client side. This should not be shared with anyone else.

```
getPublicKey: privateKey => {

const privateBuffer = _decodeHex(privateKey)

const publicKey = secp256k1.publicKeyCreate(privateBuffer)

return publicKey.toString('hex')

}
```

This function returns the public key derived from the 256-bit private key created above. This is the key that is safe to share. It takes in the 256-bit private key and returns the public key as a hex string.<br>

These two keys are stored together in the browser’s localStorage. If you do Inspect Element and navigate to the Application tab, under localStorage, you can view the key pairs.<br>

* ** **Note**: This method is not the way this would be done in a production application. **

Within the user interface, you can create these keys from the Select Holder dropdown. You can use this same dropdown to switch between multiple users in **localStorage**.

![create-a-user-sawtooth](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/5f06b8ccc63eaa2aceae22bae6dd2bc7/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Tuna_Create_New_Key_Pair.png)

## Creating a Record of Tuna

```
const createAsset = (asset, owner, state) => {

const address = getAssetAddress(asset)

return state.get([address])

.then(entries => {

const entry = entries[address]

if (entry && entry.length > 0) {

throw new InvalidTransaction('Asset name in use')

}

return state.set({

[address]: encode({name: asset, owner})

})

})

}

const getAddress = (key, length = 64) => {

return createHash('sha512').update(key).digest('hex').slice(0, length)

}

const getAssetAddress = name => PREFIX + '00' + getAddress(name, 62)
```

The **`createAsset`** function adds a new asset to the state by taking in an asset name, owner as the public key, and state. Once an asset address for a specific tuna is created with the **`sha512`** hash, the state is set to **`state.set({ [address]: encode({name: asset, owner: owner}) })`**.<br>

Within the user interface, select the owner of the tuna in the Select Holder dropdown, then provide a unique name for the tuna in the Create Tuna Record text box. Lastly, click the Create button.

![create-tuna-record1](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/ecf5dc6278ce756c419daa608a5a12c8/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Tuna_Create.png)

After doing this, you will see the tuna show up in the Tuna List, along with its associated owner.

![create-tuna-record2](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/542e69f298f8ece8eb21325c675e0fe0/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Tuna_List.png)

## Transferring a Tuna

```
const transferAsset = (asset, owner, signer, state) => {

const address = getTransferAddress(asset)

const assetAddress = getAssetAddress(asset)

return state.get([assetAddress])

.then(entries => {

const entry = entries[assetAddress]

if (!entry || entry.length === 0) {

throw new InvalidTransaction]('Asset does not exist')

}

if (signer !== decode(entry).owner) {

throw new InvalidTransaction('Only an Asset\'s owner may transfer it')

}

return state.set({

[address]: encode({name: asset, owner: owner})

})

})

}
```

The **`transferAsset`** function proposes a transfer of ownership for a particular asset to the state by taking in the asset name, owner to transfer to, signer (current owner) and state. If all the checks pass, the state is updated with the proposed transfer **`[address]: encode({asset, owner})`**.<br>

Any tuna assigned to a particular user can be transferred to another owner (public key) using the dropdowns under Transfer Tuna. Note that the transfer must be accepted by that user before it is finalized.

![transfer-tuna](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/b94d1ad03f10fbe7924dcd9341641bf7/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Tuna_Transfer.png)

In the above screenshot, we are proposing to transfer the ownership of **tuna4** from the current owner (identified by the key starting with 02) to a new owner (identified by the key starting with 03).

## Accepting or Rejecting Transfers (Part I)

```
const acceptTransfer = (asset, signer, state) => {

const address = getTransferAddress(asset)

return state.get([address])

.then(entries => {

const entry = entries[address]

if (!entry || entry.length === 0) {

throw new InvalidTransaction('Asset is not being transfered')

}

if (signer !== decode(entry).owner) {

throw new InvalidTransaction(

'Transfers can only be accepted by the new owner'

)

}

return state.set({

[address]: Buffer(0),

[getAssetAddress(asset)]: encode({name: asset, owner: signer})

})

})

}
```

The **`acceptTransfer`** function allows a user to accept a transfer of an asset and change the asset ownership.

## Accepting or Rejecting Transfers (Part II)

```
const rejectTransfer = (asset, signer, state) => {

const address = getTransferAddress(asset)

return state.get([address])

.then(entries => {

const entry = entries[address]

if (!entry || entry.length === 0) {

throw new InvalidTransaction('Asset is not being transfered')

}

if (signer !== decode(entry).owner) {

throw new InvalidTransaction(

'Transfers can only be rejected by the potential new owner')

}

return state.set({

[address]: Buffer(0)

})

})

}
```

The **rejectTransfer** function allows a user to reject a transfer of an asset, and the asset owner will remain with the original user.<br>

Within the user interface, any pending transfers for the selected user will appear under Accept Tuna. These can be accepted (with an immediate change in ownership), or rejected with the corresponding buttons.

## Accepting or Rejecting Transfers (Part III)

When we click *Transfer*, the proposal is sent to the Hyperledger Sawtooth network. If we switch the Select Holder field to the owner identified by the key starting with `03`, we will see that we can now *Accept* or *Reject* this ownership change.

* **NOTE**: If someone other than the owner tries to transfer ownership, you will see an error returned to the client from the Hyperledger Sawtooth network.

![accepting-or-rejecting-transfers](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/f1a4c52de931fb169b1685917c5d832b/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Tuna_Accept-Reject.png)



## Shutting Down Docker

**Note**: When we are done with this tutorial, run the following command to shut down Docker:

```
$ docker-compose -f sawtooth-default.yaml down
```

## Application Flow

![application-flow](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/148eb99c4e9410d799444b72fb7c6785/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/sawtooth-application-txflowdiagram.png)

Hyperledger Sawtooth allows entities to securely update and read the distributed ledger without involving a central authority. Developers create application and transaction processor business logic (smart contract).<br>

Through the client application, users (fisherman, regulator, restaurant) are able to modify the state by creating and applying transactions.<br>

Through a REST API, the client application creates a batch containing a single transaction, and submits it to the validator.<br>

The validator applies the transaction using the transaction processor, which makes a change to the state (e.g., creating a record of a tuna catch).